- 直接求解$Ax = b $,复杂度：$O(n^{3})
- 求解上三角或下三角矩阵，复杂度：$O(n^{2})
# LU分解
$$Ax = b \Rightarrow LUx = b  ( 令Ux = y ) \Rightarrow Ly = b $$
- 原本的$Ax = b$ 的计算复杂度是$O(n^{3})$
- 经过这一些列变换，计算复杂度是一个$O(n^{3})$[高斯消元法，化上三角] 和2个$O(n^{2})$ [上三角求解]

A: symmetric positive definite(SPD,对称正定),可以用$LL^{T}$

对于三对角矩阵，追赶法

In [1]:
using BenchmarkTools

In [53]:
#上三角
function back_sub(A, b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i + 1 : n
            x[i] -= A[i, j] * x[j]
        end
        x[i] = x[i] / A[i, i]
    end
    return x
end

back_sub (generic function with 1 method)

In [28]:
#下三角
function forward_sub(A,b)
    x = zeros(n)
    for i = 1:n
        x[i] = b[i]
        # println("x [$i] =" , x[i])
        for j = 1 : i-1
            x[i] -= A[i, j] * x[j]
        end
        # println("i = ",i)
        x[i] = x[i] / A[i, i]
        # println("最后的x [$i] =" , x[i])
    end
    return x
end

forward_sub (generic function with 1 method)

In [4]:
function LU_Factorization(A,b)
    n = size(A, 1)
    L = zeros(n, n)
    U = copy(A)

    for k = 1:n
        for i = k + 1 : n
            L[i,k] = U[i,k]/U[k,k]
            for j = k : n
                U[i, j] -= L[i,k] * U[k,j]
            end
        end
    end

    for i = 1:n
        L[i,i] = 1
    end
    # println(L)
    # println(U)
    # return L,U
    y = forward_sub(L,b)
    x = back_sub(U, y)
    return x
end

LU_Factorization (generic function with 1 method)

In [6]:
n = 4;
A = [1.0 1.0 2.0 3.0;0.0 2.0 1.0 2.0;1.0 -1.0 2.0 2.0;2.0 2.0 5.0 9.0];
b = [3.0; 1.0; 3.0; 7.0];
LU_Factorization(A,b)

4-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0

In [8]:
n = 3

A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
LU_Factorization(A,b)

3-element Vector{Float64}:
  0.5235569989007299
 -0.4397820146008997
  0.7782347713846534

In [10]:
using PyPlot
#随机生成n阶矩阵
n = 100
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
# figure()
# spy(A)
# display(gcf())
L,U = LU_Factorization(A,b)
# y = forward_sub(L,b)
# x = back_sub(U, y)

100-element Vector{Float64}:
  0.37035290277421085
 -0.6568298042222591
 -0.8586670005450586
 -1.2400468499437471
  1.9947915461430563
  0.6161142226137745
  1.6338165334602988
 -1.9936180232231162
 -3.1117674178785606
  1.2392475154852354
  ⋮
  0.5302296336515686
  0.7114047587249877
  4.286547327796872
 -3.356690400153187
  1.8116340300731333
  1.1623471362401314
 -0.569534935617437
  1.1041825516298898
 -2.2531916335505557

In [11]:
A\b

100-element Vector{Float64}:
  0.3703529027736924
 -0.6568298042229409
 -0.8586670005450632
 -1.2400468499445407
  1.9947915461441685
  0.6161142226140076
  1.6338165334613401
 -1.9936180232241725
 -3.1117674178801282
  1.2392475154862959
  ⋮
  0.5302296336517407
  0.7114047587251902
  4.286547327798877
 -3.3566904001548123
  1.8116340300744433
  1.162347136240845
 -0.5695349356181254
  1.1041825516305124
 -2.253191633551273

第四次作业
- 写一个lineSolver.jl文件（截图发邮箱）
- drawio.com，

In [12]:
#三对角矩阵求解（追赶法）
function tri_DiagSolver(A,b)
    n = size(A, 1)
    u = zeros(n)
    y = zeros(n)

    u[1] = A[1,2]/A[1,1]
    y[1] = b[1]/A[1,1]
    for i = 2:n-1
        u[i] = A[i,i+1]/(A[i,i]-u[i-1]*A[i,i-1])
        y[i] = ((b[i]-y[i-1]*A[i,i-1])/(A[i,i]-u[i-1]*A[i,i-1]))        
    end
    y[n] = ((b[n]-y[n-1]*A[n,n-1])/(A[n,n]-u[n-1]*A[n,n-1]))
    # 回代求解x
    x = zeros(n)
    x[n] = y[n]
    for i = n-1:-1:1
        x[i] = y[i] - u[i] * x[i+1]
    end
    return x
end

tri_DiagSolver (generic function with 1 method)

In [13]:
A = [1.0 2.0 0.0; 2.0 1.0 1.0; 0.0 1.0 1.0]
b = [1.0, 2.0, 3.0]
tri_DiagSolver(A,b)

3-element Vector{Float64}:
 -0.5
  0.75
  2.25

In [17]:
#随机生成三对角矩阵
using PyPlot

n = 50
A = zeros(n, n)
for i = 1:n
    A[i, i] = randn(1)[1]
    if i>1
        A[i, i-1] = randn(1)[1] 
    end
    if i<n
        A[i,i+1] = randn(1)[1]
    end
end
b = randn(n);
# println(A)
# figure()
# spy(A)
# display(gcf())
tri_DiagSolver(A,b)

50-element Vector{Float64}:
   6.1070331796441035
  -1.5260351901202762
   4.454593615074032
   1.3218320729773518
   0.05025237289068896
   0.4645053878955822
  -1.0131772190537394
   0.959299596558881
  -0.29287305963523824
   3.0831229411904473
   ⋮
   5.1177050911276645
   0.17193902880039924
  -1.8089570504827486
  -0.6251153546316126
  -4.339173928695723
  -3.184438926185806
   4.134879799999258
 -11.63917288455576
   2.1500080955781002

In [18]:
A\b

50-element Vector{Float64}:
   6.107033179644105
  -1.526035190120277
   4.454593615074024
   1.321832072977352
   0.050252372890691566
   0.4645053878955815
  -1.0131772190537371
   0.9592995965588813
  -0.2928730596352387
   3.083122941190447
   ⋮
   5.117705091127661
   0.1719390288003986
  -1.8089570504827481
  -0.6251153546316124
  -4.339173928695722
  -3.184438926185805
   4.134879799999257
 -11.639172884555764
   2.1500080955781002

In [4]:
n = 3
A = zeros(n,n)
for i in 1:n
    A[i,i] = 1
end
A

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [64]:
#LDL分解求解器
function LDL(A,b)   
    n = size(A,1)
    L = copy(A)
    D = zeros(n,n)
    Lt = zeros(n,n)
    y = copy(b)
    x = zeros(n)

    #给L和D赋值
    for i = 1:n
        for j = i:n
            if i == j
                L[i,j] = 1
                D[i,j] = A[i,i]
            else
                L[i,j] = 0
            end
        end
    end

    #求解L和D
    for i = 1:n
        for j = 1:i-1
            for k =1:j-1
                L[i,j] = L[i,j] - D[k,k]*L[i,k]*L[j,k]
            end 
            L[i,j] = L[i,j]/D[j,j]
        end

        for j = 1:i-1
            D[i,i] -= D[j,j] * L[i,j]^2
        end 
    end

    println("L:",L)
    println("D:",D)
    
    y = forward_sub(L,b)
    println("y = ", y)
    # for i =1:n
    #     for k = 1:n-1 
    #         y[i] -= L[i,k]*y[k] 
    #     end 
    # end
    
    #求解x
    for i = n:-1:1
        x[i] = y[i]/D[i,i]
        
        for j = i + 1 : n
            x[i] -= L[j, i] * x[j]
        end
        println("x[$i] = ",x[i])
    end

    #L转置
    # for i = 1:n
    #     for j = i:n 
    #         if i == j
    #             Lt[i,j] = 1
    #         else
    #             Lt[i,j] = L[j,i]
    #         end
    #     end
    # end
    # println(Lt)
    # return L,D
    return x
end

LDL (generic function with 1 method)

In [79]:
#LDL分解求解器2
function LDL2(A,b)   
    n = size(A,1)
    L = copy(A)
    D = zeros(n,n)
    Lt = zeros(n,n)
    y = copy(b)
    x = zeros(n)

    #给L和D赋值
    for i = 1:n
        for j = i:n
            if i == j
                L[i,j] = 1
                D[i,j] = A[i,i]
            else
                L[i,j] = 0
            end
        end
    end

    #求解L和D
    for i = 1:n
        for j = 1:i-1
            for k =1:j-1
                L[i,j] = L[i,j] - D[k,k]*L[i,k]*L[j,k]
            end 
            L[i,j] = L[i,j]/D[j,j]
        end

        for j = 1:i-1
            D[i,i] -= D[j,j] * L[i,j]*L[i,j]
        end 
    end

    println("L = ",L)
    println("D = ",D)
    
    for i = 1:n
        y[i] = b[i]
        for j in 1:i-1
            y[i] -= L[i,j]*y[j]
        end
        println("y[$i] = ", y[i]) 
    end
    

    #L转置
    for i = 1:n
        for j = i:n 
            if i == j
                Lt[i,j] = 1
            else
                Lt[i,j] = L[j,i]
            end
        end
    end
    println("Lt = ",Lt)
    M = zeros(n)
    for i =  1:n
        M[i] = y[i]/D[i,i]
    end
    println("M = ",M)
    
    x = back_sub(Lt,M)
    return x
end

LDL2 (generic function with 1 method)

In [97]:
A\b

3-element Vector{Float64}:
 0.4324324324324324
 0.5405405405405406
 1.864864864864865

In [98]:
n = 3
A = [4.0 1.0 2.0; 1.0 5.0 1.0; 2.0 1.0 3.0]
b = [6.0;5.0;7.0]
x1 = LDL(A,b)
# x = LDL(A,b)

L:[1.0 0.0 0.0; 0.25 1.0 0.0; 0.5 0.10526315789473684 1.0]
D:[4.0 0.0 0.0; 0.0 4.75 0.0; 0.0 0.0 1.9473684210526316]
y = [6.0, 3.5, 3.6315789473684212]
x[3] = 1.864864864864865
x[2] = 0.5405405405405406
x[1] = 0.43243243243243246


3-element Vector{Float64}:
 0.43243243243243246
 0.5405405405405406
 1.864864864864865

In [74]:
n = 3
A = [4.0 1.0 2.0; 1.0 5.0 1.0; 2.0 1.0 3.0]
b = [6.0;5.0;7.0]
L,D,Lt = LDL2(A,b)
C = L*D*Lt
D = L*Lt
println("C =",C)
println("D = ",D)

L = [1.0 0.0 0.0; 0.25 1.0 0.0; 0.5 0.10526315789473684 1.0]
D = [4.0 0.0 0.0; 0.0 4.75 0.0; 0.0 0.0 1.9473684210526316]
y[1] = 6.0
y[2] = 3.5
y[3] = 3.6315789473684212
Lt = [1.0 0.25 0.5; 0.0 1.0 0.10526315789473684; 0.0 0.0 1.0]
M = [1.5, 0.7368421052631579, 1.864864864864865]
C =[4.0 1.0 2.0; 1.0 5.0 1.0; 2.0 1.0 3.0]
D = [1.0 0.25 0.5; 0.25 1.0625 0.23026315789473684; 0.5 0.23026315789473684 1.2610803324099722]


In [ ]:
M = zeros(n)
for i =  1:n
    M[i] = y[i]/D[i,i]
end


In [70]:
A = [4.0 12.0 -16.0; 12.0 37.0 -43.0; -16.0 -43.0 98.0]
LDL(A,b)

L:[1.0 0.0 0.0; 3.0 1.0 0.0; -4.0 5.0 1.0]
D:[4.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 9.0]
y = [6.0, -13.0, 96.0]
x[3] = 10.666666666666666
x[2] = -66.33333333333333
x[1] = 243.16666666666666


3-element Vector{Float64}:
 243.16666666666666
 -66.33333333333333
  10.666666666666666

In [93]:
n = 3
A = [4.0 -2.0 4.0; -2.0 17.0 10.0; 4.0 10.0 9.0]
b = [10.0 ;3.0;-7.0]
x1 = LDL2(A,b)
x2 = LDL(A,b)
println("x1=",x1)
println("x2 = ",x2)


L = [1.0 0.0 0.0; -0.5 1.0 0.0; 1.0 0.75 1.0]
D = [4.0 0.0 0.0; 0.0 16.0 0.0; 0.0 0.0 -4.0]
y[1] = 10.0
y[2] = 8.0
y[3] = -23.0
Lt = [1.0 -0.5 1.0; 0.0 1.0 0.75; 0.0 0.0 1.0]
M = [2.5, 0.5, 5.75]
L:[1.0 0.0 0.0; -0.5 1.0 0.0; 1.0 0.75 1.0]
D:[4.0 0.0 0.0; 0.0 16.0 0.0; 0.0 0.0 -4.0]
y = [10.0, 8.0, -23.0]
x[3] = 5.75
x[2] = -3.8125
x[1] = -5.15625
x1=[-5.15625, -3.8125, 5.75]
x2 = [-5.15625, -3.8125, 5.75]


In [94]:
A\b

3-element Vector{Float64}:
 -5.15625
 -3.8125
  5.75